# Extraccion de datos no estructurados
de un archivo CSV usando `PydanticOutputParser`

Imports

In [6]:
import pandas as pd
import json
import unicodedata

from typing import Optional, Sequence
from langchain.output_parsers import PydanticOutputParser
from transformers import pipeline
from pydantic import BaseModel, Field

carga de datos en un Dataframe

In [7]:
# Carga de datos desde un CSV
df = pd.read_csv("df_combined.csv", sep=',', encoding='utf-8')

In [8]:
df.head()

,id,institucionDependencia,CVE_ENT,Tipo_organismo_poder,Funcionalidad de la institución,servidorPublicoSancionado,Sexo,resolucion,inhabilitacion,fechaCaptura,...,autoridadSancionadora,tipoSancion,Causa_motivo_hechos,observaciones,Puesto,Nivel_salarial,Nivel_mando,salario_estimado,meses_inhabil,Nivel_mando2
0,630d3b3390bf7b1af7b620f6,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,24,Organismo público descentralizado,Seguridad Social,"{'nombres': 'ZULAIDA', 'primerApellido': 'SILV...",1,{'fechaResolucion': '06/08/2018'},"{'fechaInicial': '06/08/2018', 'fechaFinal': '...",31/01/2019,...,ORGANO INTERNO DE CONTROL,INHABILITADO,NEGLIGENCIA ADMINISTRATIVA,NO REALIZ� UNA ADECUADA DEFENSA JUR�DICA DENTR...,SUBJEFE DE DEPARTAMENTO,8,Personal de enlace,17919,120.0,3
1,630d3b3390bf7b1af7b620f7,GOBIERNO DE LA CIUDAD DE MEXICO,9,Gobierno estatal,Gobierno,"{'nombres': 'YONATAN', 'primerApellido': 'SALA...",0,{'fechaResolucion': '27/06/2013'},"{'fechaInicial': '27/06/2013', 'fechaFinal': '...",18/05/2015,...,CONTRALORIA DE ESTADO,INHABILITADO,NEGLIGENCIA ADMINISTRATIVA,NO LOS PRECISAN.,RESIDENTE DE OBRA. SR�A. DEOBRAS Y SERVICIOS.,8,Personal de enlace,17919,132.0,3
2,630d3b3390bf7b1af7b620f8,GOBIERNO DEL ESTADO DE PUEBLA,21,Gobierno estatal,Gobierno,"{'nombres': 'YOLANDA', 'primerApellido': 'CORT...",1,{'fechaResolucion': '11/06/2018'},"{'fechaInicial': '11/06/2018', 'fechaFinal': '...",18/07/2018,...,JUZGADOS DE DISTRITO,INHABILITADO,DELITO COMETIDO POR SERVIDORES PUBLICOS,"YOLANDA CORT�S BALTAZAR, ES PENALMENTE RESPONS...",REGIDOR DEL CABILDO MUNICIPAL DEL AYUNTAMIENTO...,10,enlace_oper_categoria,27000,40.0,3
3,630d3b3390bf7b1af7b620f9,GOBIERNO DEL ESTADO DE GUANAJUATO,11,Gobierno estatal,Gobierno,"{'nombres': 'YOLANDA', 'primerApellido': 'CHAV...",1,{'fechaResolucion': '29/02/2016'},"{'fechaInicial': '29/02/2016', 'fechaFinal': '...",15/06/2017,...,CONTRALORIA MUNICIPAL,INHABILITADO,ABUSO DE AUTORIDAD,DUPLICIDAD DE CARGOS PUBLICOS EN EL JUZGADO AD...,SECRETARIA,8,Personal de enlace,17919,120.0,3
4,630d3b3390bf7b1af7b620fa,GOBIERNO DEL ESTADO DE HIDALGO,13,Gobierno estatal,Gobierno,"{'nombres': 'YOLANDA', 'primerApellido': 'TELL...",1,{'fechaResolucion': '04/12/2020'},"{'fechaInicial': '08/12/2020', 'fechaFinal': '...",08/12/2020,...,CONTRALORIA DE ESTADO,INHABILITADO,NEGLIGENCIA ADMINISTRATIVA,"DEPENDENCIA: MUNICIPIO DE PACHUCA, HIDALGO. \r...",EX PRESIDENTA MUNICIPAL,5,Presidencia municipal,45000,12.0,2


In [9]:
# Convertir una columna en diccionarios
def expand_json_column(json_string):
    try:
        # Reemplazar comillas simples por dobles y eliminar espacios extra
        clean_json = json_string.strip().replace("'", '"')
        # Convertir a un diccionario
        return json.loads(clean_json)
    except json.JSONDecodeError:
        return {}  # Devuelve un diccionario vacío si hay un error

# Función para limpiar y corregir caracteres especiales mientras se preservan los acentos
def clean_text_preserve_accents(text):
    try:
        # Convertir a texto (str) y normalizar para corregir caracteres especiales
        text = str(text)
        text = unicodedata.normalize('NFKD', text).encode('latin-1', 'ignore').decode('utf-8')
        return text
    except Exception as e:
        return text  # Devuelve el texto original si ocurre algún error

In [10]:
# Aplicar la función a la columna 'info'
expanded_data = df['servidorPublicoSancionado'].apply(expand_json_column)
# Crear nuevas columnas a partir de los datos expandidos
expanded_df = pd.json_normalize(expanded_data)
# Combinar el DataFrame original con las nuevas columnas
df = pd.concat([df, expanded_df], axis=1)
# Eliminar la columna original si ya no es necesaria
df = df.drop(columns=['servidorPublicoSancionado'])

# Aplicar la función a la columna 'info'
expanded_data = df['resolucion'].apply(expand_json_column)
# Crear nuevas columnas a partir de los datos expandidos
expanded_df = pd.json_normalize(expanded_data)
# Combinar el DataFrame original con las nuevas columnas
df = pd.concat([df, expanded_df], axis=1)
# Eliminar la columna original si ya no es necesaria
df = df.drop(columns=['resolucion'])

# Aplicar la función a la columna 'info'
expanded_data = df['inhabilitacion'].apply(expand_json_column)
# Crear nuevas columnas a partir de los datos expandidos
expanded_df = pd.json_normalize(expanded_data)
# Combinar el DataFrame original con las nuevas columnas
df = pd.concat([df, expanded_df], axis=1)
# Eliminar la columna original si ya no es necesaria
df = df.drop(columns=['inhabilitacion'])

# Aplicar la limpieza a la columna 'descripcion'
df['observaciones'] = df['observaciones'].apply(clean_text_preserve_accents)
df['puesto'] = df['puesto'].apply(clean_text_preserve_accents)

df['observaciones'] = df['observaciones'].astype(str)
df['puesto'] = df['puesto'].astype(str)

In [11]:
df.head()

,id,institucionDependencia,CVE_ENT,Tipo_organismo_poder,Funcionalidad de la institución,Sexo,fechaCaptura,expediente,autoridadSancionadora,tipoSancion,...,meses_inhabil,Nivel_mando2,nombres,primerApellido,segundoApellido,puesto,fechaResolucion,fechaInicial,fechaFinal,plazo
0,630d3b3390bf7b1af7b620f6,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,24,Organismo público descentralizado,Seguridad Social,1,31/01/2019,000930/2015,ORGANO INTERNO DE CONTROL,INHABILITADO,...,120.0,3,ZULAIDA,SILVA,SOLORIO,SUBJEFE DE DEPARTAMENTO,06/08/2018,06/08/2018,06/08/2028,Dato no proporcionado
1,630d3b3390bf7b1af7b620f7,GOBIERNO DE LA CIUDAD DE MEXICO,9,Gobierno estatal,Gobierno,0,18/05/2015,CI/SOS/A/0083/2012,CONTRALORIA DE ESTADO,INHABILITADO,...,132.0,3,YONATAN,SALAS,ALARCON,RESIDENTE DE OBRA. SRA. DEOBRAS Y SERVICIOS.,27/06/2013,27/06/2013,23/06/2024,Dato no proporcionado
2,630d3b3390bf7b1af7b620f8,GOBIERNO DEL ESTADO DE PUEBLA,21,Gobierno estatal,Gobierno,1,18/07/2018,PUE/51/2018 CAUSA PENAL,JUZGADOS DE DISTRITO,INHABILITADO,...,40.0,3,YOLANDA,CORTES,BALTAZAR,REGIDOR DEL CABILDO MUNICIPAL DEL AYUNTAMIENTO...,11/06/2018,11/06/2018,10/10/2021,Dato no proporcionado
3,630d3b3390bf7b1af7b620f9,GOBIERNO DEL ESTADO DE GUANAJUATO,11,Gobierno estatal,Gobierno,1,15/06/2017,PRA-02/2015,CONTRALORIA MUNICIPAL,INHABILITADO,...,120.0,3,YOLANDA,CHAVEZ,CENTENO,SECRETARIA,29/02/2016,29/02/2016,25/02/2026,Dato no proporcionado
4,630d3b3390bf7b1af7b620fa,GOBIERNO DEL ESTADO DE HIDALGO,13,Gobierno estatal,Gobierno,1,08/12/2020,PRA/015/2020,CONTRALORIA DE ESTADO,INHABILITADO,...,12.0,2,YOLANDA,TELLERIA,BELTRAN,EX PRESIDENTA MUNICIPAL,04/12/2020,08/12/2020,08/12/2021,Dato no proporcionado


Crear una estrcutra de datos Pydantic para el caso de uso

In [12]:
# Modelos Pydantic para el caso de negocio inteligencia competitiva
class Sancionados(BaseModel):
    delito: str = Field(description="tipo de delito jurídico")
    resolucion: str = Field(description="tipo de castigo imputado")
    ubicacion: str = Field(description="lugar donde se aborda el caso")
    productos_servicios: str = Field(description="producto o servicio del que deriva el delito")
    detalles_adicionales: str = Field(description="cualquier otro detalle sobre el delito")

class Delito(BaseModel):
    """Identificar información sobre cada competidor"""
    persona: Sequence[Sancionados]

# Preguntas para extraer características específicas
preguntas = {
    "delito": "¿Es un tipo de delito sexual? Si es así identifica el delito",
    "resolucion": "¿Cuál es el castigo imputado al delito identificado?",
    "ubicacion": "¿En que estado de la república mexicana se determinó la resolución?",
    "productos_servicios": "¿Qué producto o servicio fue objeto del delito?",
    "detalles_adicionales": "¿Qué otros detalles relevantes se mencionan sobre el delito?"
}

# Inicializamos el modelo de Hugging Face para tareas de extracción de información
extractor = pipeline("question-answering", model="timpal0l/mdeberta-v3-base-squad2")

# Configuracion del parser Pydantic y prompt template
parser = PydanticOutputParser(pydantic_object=Delito)

C:\Users\jesla\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Función para procesar cada fila de datos y extraer informacion

In [17]:
# Función para procesar cada fila de datos y extraer informacion
def process_row(row):
    respuestas = {}
    for campo, pregunta in preguntas.items():
      # Convert 'observaciones' to string to ensure compatibility
        context = str(row['observaciones'])

        respuesta = extractor(question=pregunta, context= context)
        respuestas[campo] = respuesta['answer']

    persona_data = [Sancionados(**respuestas)]
    sancionado_data = Delito(persona=persona_data)
    json_data = sancionado_data.json()

    result = parser.parse(json_data)

    # Convert Pydantic result to a dictionary
    persona_data = result.model_dump()

    # Flatten the nested structure for DataFrame creation
    flat_data = {'CVE_ENT': [], 'delito': [], 'resolucion': [], 'ubicacion': [], 'productos_servicios': [], 'detalles_adicionales': []}

    for entry in persona_data['persona']:
        flat_data['CVE_ENT'].append(row['CVE_ENT'])
        flat_data['delito'].append(entry['delito'])
        flat_data['resolucion'].append(entry['resolucion'])
        flat_data['ubicacion'].append(entry['ubicacion'])
        flat_data['productos_servicios'].append(entry['productos_servicios'])
        flat_data['detalles_adicionales'].append(entry['detalles_adicionales'])

    # Create a DataFrame from the flattened data
    df_delito = pd.DataFrame(flat_data)

    return df_delito

Aplica la funcion a cada fila y concatena los resultados

In [18]:
# Aplica la funcion a cada fila y concatena los resultados
intel_df = pd.concat(df.apply(process_row, axis=1).tolist(), ignore_index=True)

Muestra los datos obtenidos

In [19]:
# Despliega el DataFrame con datos estructurados
intel_df.head(5)

,CVE_ENT,delito,resolucion,ubicacion,productos_servicios,detalles_adicionales
0,24,NO REALIZ UNA ADECUADA DEFENSA JURDICA,NO REALIZ UNA ADECUADA DEFENSA JURDICA,SAN LUIS POTOS,NO REALIZ UNA ADECUADA DEFENSA JURDICA,NO REALIZ UNA ADECUADA DEFENSA JURDICA
1,9,NO LOS PRECISAN.,NO LOS PRECISAN.,NO LOS PRECISAN.,NO LOS PRECISAN.,NO LOS PRECISAN.
2,21,A TTULO DOLOSO,SENTENCIA DE AMPARO.,"PUEBLA,",SERVIDOR PBLICO,"PRECEPTOS 9, PRRAFO PRIMERO Y 13, FRACCIN III,"
3,11,DUPLICIDAD DE CARGOS PUBLICOS,DUPLICIDAD DE CARGOS PUBLICOS,\r\nAUTORIDAD SANCIONADORA: SALVATIERRA.,"EMPLEO, CARGO O COMISIN PBLICO,",DUPLICIDAD DE CARGOS PUBLICOS
4,13,ALS NORMAS SEALAN PARA TALES EFECTOS.,INDEBIDA Y ARBITRARIA,"PACHUCA DE SOTO, HIDALGO.",LA INDEBIDA Y ARBITRARIA CREACION,TODOS LOS EFECTOS LEGALES


In [20]:
intel_df.to_csv('anticorrupcion_intel.csv', index=False)

In [21]:
intel_df.to_pickle('anticorrupcion.pkl')